In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

#### Group 1: All variables still collected after 2023 (excluding information on diseases)

##### Read data

In [ ]:
data_vs= pd.read_csv('data.csv',index_col=False).drop(columns=['koosChaSco', 'funcChaSco'])
print("data shape:", data_vs.shape)
print("number of missing values:", data_vs.isna().sum().sum())

In [ ]:
# Group 1: All variables still collected after 2023 (excluding information on diseases) 
group1 = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", 
          "ptb_eq5d_vas", "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", 
          "ptb_koospain", "fysb_9349", "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", 
          "ptb_3762", "ptb_13118", "fysb_medicin", "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b", 
          "vasChaSco"]
print("group1:", len(group1))

##### Desciption Of VAS

In [ ]:
description = data_vs['vasChaSco'].describe()
description

In [ ]:
average_reduction = np.mean(data_vs.vasChaSco)
average_reduction

##### Feature selection

In [ ]:
###########  ---------------------- Function for create list of feature selection --------------------- ###############
def feature_selection(X_train, X_test, y_train, y_test, n_features):
    
    # 1. Fit a random forest regression model to the training set
    rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)


    # 2.Get the feature importances and select the top k features
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]

    k = n_features
    selected_indices = indices[:k]
    selected_features = X_train.columns[selected_indices]
    print(selected_features)

    # 3. Train a new random forest regression model using only the selected features
    rf_selected = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf_selected.fit(X_train[selected_features], y_train)

    # 4. Evaluate the performance of the new model on the testing set
    y_pred = rf_selected.predict(X_test[selected_features])
    mse = mean_squared_error(y_pred, y_test)
    rmse = mse ** 0.5
    rsquared = rf_selected.score(X_test[selected_features], y_test)

    print(mean_squared_error(y_pred, y_test))
    print(f"RMSE: {rmse:.2f}")
    print(f"R-squared: {rsquared:.2f}")
    
    return selected_features, rmse, rsquared

##### Cross validation

In [ ]:
###########  ---------------------- Cross Validation --------------------- ###############
# Feature selection, train model with cross validation
def feature_selection_kfold(data, num_features=34, target_column='vasChaSco', num_folds=10, random_state=42, n_features=11):
    features = data.iloc[:, :num_features]
    y = data[target_column]
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    model = RandomForestRegressor()
    mse_scores = []
    list_select_features = []
    rmse_list = []
    rsquared_list = []
    for train_index, val_index in kf.split(features):
        X_train, y_train = features.iloc[train_index], y.iloc[train_index]
        X_test, y_test = features.iloc[val_index], y.iloc[val_index]
        list_features, rmse_feature, rsquared_feature = feature_selection(X_train, X_test, y_train.values, y_test.values, n_features=n_features)
        list_select_features.append(list_features)
        rmse_list.append(rmse_feature)
        rsquared_list.append(rsquared_feature)
    print("------------------------------------------------------------")
    print("RMSE list: ", rmse_list) 
    print('\n')
    print("RSquared list: ", rsquared_list)
    print('\n')
    print("Mean RMSE in k-fold: ", np.mean(rmse_list))
    print("Mean RSquared in k fold: ", np.mean(rsquared_list))
    
    return list_select_features, rmse_list, rsquared_list

In [ ]:
list_select_features, rmse_list, rsquared_list = feature_selection_kfold(data_vs, num_features=34, target_column='vasChaSco', num_folds=10, random_state=42)

##### Feature selection 11 Top variables

In [ ]:
# How many times every feature repeat ?
dict_features = {}
for i in list_select_features:
    for j in i:
        if j in dict_features:
            dict_features[j]+=1  
        else:
            dict_features[j] = 1  
print (dict_features)

##### Gini - variables importance

In [ ]:
def get_feature_importances(data, target_column, num_features=34):

    features = data.iloc[:, :num_features]
    y = data[target_column]
    

    rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(features, y)
    

    importances = rf.feature_importances_
    

    feature_importance_df = pd.DataFrame({
        'Variable': features.columns,
        'Importance': importances
    })
    

    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    
    return feature_importance_df


feature_importance_df = get_feature_importances(data_vs, target_column='vasChaSco')


all_feaatures = feature_importance_df
top_15_features = feature_importance_df.head(15)
print(all_feaatures)
print("------------------------------------------------------------")
print(top_15_features)

In [ ]:
feature = [
"ptb_3764","ptb_eq5d_score_5l","testb_10391","fysb_symptomvarighed","fysb_BMI",
"ptb_eq5d_vas","age","ptb_koos_qol_score","testb_10281","pain_area","ptb_4145",
]

In [ ]:
def plot_all_feature_importances(data, target_column):
    
    features = data.drop(columns=[target_column])
    y = data[target_column]
    
   
    rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(features, y)
    

    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]
    

    sorted_importances = importances[indices]
    sorted_features = features.columns[indices]
    

    plt.figure(figsize=(12, min(0.25 * len(sorted_features), 20)))
    sns.barplot(x=sorted_importances, y=sorted_features)
    plt.title('Random forest regressor Variable Importances using Gini Impurity for VAS pain change score')
    plt.xlabel('Relative Importance')
    plt.ylabel('Variables')
    plt.show()


    feature_importance_df = pd.DataFrame({
        'Feature': sorted_features,
        'Importance': sorted_importances
    })
    return feature_importance_df


feature_importance_df = plot_all_feature_importances(data_vs, target_column='vasChaSco')


##### Elbow

In [ ]:
avg_rmse = []
avg_rsquared = []
for k in range(1,34):
    list_select_features, rmse_list, rsquared_list = feature_selection_kfold(data_vs, num_features=34, target_column='vasChaSco', num_folds=10, random_state=42, n_features=k)
    avg_rmse.append(np.average(rmse_list))
    avg_rsquared.append(np.average(rsquared_list))

In [ ]:
plt.scatter(range(1, 34), avg_rmse)
plt.xlabel('Number of variables')
plt.ylabel('Average RMSE')
plt.title('RMSE vs Number of variables')
plt.show()

In [ ]:
plt.plot(range(1, 34), avg_rsquared)
plt.xlabel('Number of variables')
plt.ylabel('Average R-squared')
plt.title('R-squared vs Number of Variables')
plt.show()

#### Clinical relevent

#### different interval (with all variables and other categorizing)

In [ ]:
def calculate(avg_prediction, personilize_prediction, y_test, interval):
    upperBound = y_test + interval
    lowerBound = y_test - interval
    number_samples = len(y_test)
    
    personilized_prediction_inside_interval = np.logical_and(lowerBound <= personilize_prediction, personilize_prediction <= upperBound)
    avg_prediction_inside_interval = np.logical_and(lowerBound <= avg_prediction, avg_prediction <= upperBound)
    
    return (np.sum(personilized_prediction_inside_interval)/number_samples * 100, np.sum(avg_prediction_inside_interval)/number_samples * 100)

def clinical_relevance(data, column_names, target_column='vasChaSco', num_folds=10, random_state=42, intervals=[5, 10, 15, 20]):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    features = data[column_names]
    y = data[target_column]
    detailed_results = []

    for interval in intervals:
        fold_results = []
        for fold, (train_index, val_index) in enumerate(kf.split(features)):
            X_train, y_train = features.iloc[train_index], y.iloc[train_index]
            X_test, y_test = features.iloc[val_index], y.iloc[val_index]
            rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
            rf.fit(X_train, y_train)
            
            personilize_prediction = rf.predict(X_test)
            avg_prediction = np.mean(y_train)
            
            personal_percent, avg_percent = calculate(avg_prediction, personilize_prediction, y_test, interval)
            fold_results.append({
                'fold': fold + 1,
                'personalized_percent': personal_percent,
                'avg_percent': avg_percent
            })
        
        interval_avg_personalized = np.mean([fr['personalized_percent'] for fr in fold_results])
        interval_avg_avg = np.mean([fr['avg_percent'] for fr in fold_results])
        detailed_results.append({
            'interval': interval,
            'fold_results': fold_results,
            'interval_avg_personalized': interval_avg_personalized,
            'interval_avg_avg': interval_avg_avg
        })
    
    return detailed_results


In [ ]:
# Full Model:
all_34variables = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", 
          "ptb_eq5d_vas", "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", 
          "ptb_koospain", "fysb_9349", "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", 
          "ptb_3762", "ptb_13118", "fysb_medicin", "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b"]

detailed_results = clinical_relevance(data_vs, all_34variables, target_column='vasChaSco', num_folds=10, random_state=42)


for result in detailed_results:
    print(f"Interval: {result['interval']}")
    print(f"Overall Average Personalized Prediction Inside Interval: {result['interval_avg_personalized']}%")
    print(f"Overall Average Prediction Inside Interval: {result['interval_avg_avg']}%")
    print("Fold-wise Results:")
    for fold_result in result['fold_results']:
        print(f"  Fold {fold_result['fold']}: Personalized {fold_result['personalized_percent']}%, Average {fold_result['avg_percent']}%")
    print('-------------------------------------------')

In [ ]:
# Continiues Model:
all_11variables = ["ptb_3764", "ptb_eq5d_score_5l", "testb_10391", "fysb_symptomvarighed", "fysb_BMI", "ptb_eq5d_vas", 
                   "age", "ptb_koos_qol_score", "testb_10281", "pain_area", "ptb_4145"]

detailed_results = clinical_relevance(data_vs, all_11variables, target_column='vasChaSco', num_folds=10, random_state=42)


for result in detailed_results:
    print(f"Interval: {result['interval']}")
    print(f"Overall Average Personalized Prediction Inside Interval: {result['interval_avg_personalized']}%")
    print(f"Overall Average Prediction Inside Interval: {result['interval_avg_avg']}%")
    print("Fold-wise Results:")
    for fold_result in result['fold_results']:
        print(f"  Fold {fold_result['fold']}: Personalized {fold_result['personalized_percent']}%, Average {fold_result['avg_percent']}%")
    print('-------------------------------------------')

In [ ]:
# Manageable Model:
all_6variables = ['age', 'fysb_BMI','ptb_3764','fysb_symptomvarighed', 'testb_10391', 'ptb_eq5d_score_5l']

detailed_results = clinical_relevance(data_vs, all_6variables, target_column='vasChaSco', num_folds=10, random_state=42)


for result in detailed_results:
    print(f"Interval: {result['interval']}")
    print(f"Overall Average Personalized Prediction Inside Interval: {result['interval_avg_personalized']}%")
    print(f"Overall Average Prediction Inside Interval: {result['interval_avg_avg']}%")
    print("Fold-wise Results:")
    for fold_result in result['fold_results']:
        print(f"  Fold {fold_result['fold']}: Personalized {fold_result['personalized_percent']}%, Average {fold_result['avg_percent']}%")
    print('-------------------------------------------')

##### New evaluation with all variables and different category

In [ ]:
def calculate(avg_prediction, personilize_prediction, y_test):
    intervals = [5, 10, 15, 20]
    num_samples = len(y_test)

    personilized_percentage = []
    avg_percentage = []

    for interval in intervals:
        upper_bound = y_test + interval
        lower_bound = y_test - interval
        
        personilized_prediction_inside_interval = np.logical_and(lower_bound <= personilize_prediction, personilize_prediction <= upper_bound)
        avg_prediction_inside_interval = np.logical_and(lower_bound <= avg_prediction, avg_prediction <= upper_bound)
        
        personilized_percentage.append(np.sum(personilized_prediction_inside_interval) / num_samples * 100)
        avg_percentage.append(np.sum(avg_prediction_inside_interval) / num_samples * 100)

    return personilized_percentage, avg_percentage


def clinical_relevance(data, column_names, target_column='vasChaSco', num_folds=10, random_state=42):
    
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    features = data[column_names]
    y = data[target_column]
    
    personilized_percentages = []
    avg_percentages = []
    
    for train_index, val_index in kf.split(features):

        X_train, y_train = features.iloc[train_index], y.iloc[train_index]
        X_test, y_test = features.iloc[val_index], y.iloc[val_index]
        
        rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
        rf.fit(X_train, y_train)
        
   
        personilize_prediction = rf.predict(X_test)
        avg_prediction = np.mean(y_train)
        
        personilized_percentage, avg_percentage = calculate(avg_prediction, personilize_prediction, y_test)
        personilized_percentages.append(personilized_percentage)
        avg_percentages.append(avg_percentage)

    return personilized_percentages, avg_percentages

def create_plot(data_vs, column_names):

    personilized_percentages, avg_percentages = clinical_relevance(data_vs, column_names, target_column='vasChaSco', num_folds=10, random_state=42)


    intervals = [5, 10, 15, 20]
    plt.plot(intervals, np.mean(personilized_percentages, axis=0), label='Personalized Prediction')
    plt.plot(intervals, np.mean(avg_percentages, axis=0), label='Average Prediction')
    plt.xlabel('Deviation allowed')
   
    plt.ylabel('Predictions')
    plt.title('Percentages of correct predictions with different deviations allowed')
    plt.legend()
   
    plt.ylim(0, 100)
    plt.yticks(np.arange(0, 101, 10))
    plt.xticks([5, 10, 15, 20]) 
    plt.show()

##### Also evaluation

##### for mean model and our model

In [ ]:
def evaluation_model(y_pred, y_true):
        mse = mean_squared_error(y_pred, y_true)
        rmse = mse ** 0.5
        rsquared = r2_score(y_true, y_pred)
        return mse, rmse, rsquared

    
    
def kfold_training(data, selected_features, target_column='vasChaSco', num_folds=10, random_state=42 ):
    features = data[selected_features]
    y = data[target_column]
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    model = RandomForestRegressor()
    mse_scores = []
    rmse_list = []
    rsquared_list = []
    
    mse_scores_avg = []
    rmse_list_avg = []
    rsquared_list_avg = []
    for train_index, val_index in kf.split(features):
     
        X_train, y_train = features.iloc[train_index], y.iloc[train_index]
        X_test, y_test = features.iloc[val_index], y.iloc[val_index]
        rf_selected = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
        rf_selected.fit(X_train, y_train)
        y_pred = rf_selected.predict(X_test)
        

        mse, rmse, rsquared = evaluation_model(y_pred, y_test)
      
        mse_scores.append(mse)
        rmse_list.append(rmse)
        rsquared_list.append(rsquared)
        
        # average model
        avg_prediction = np.mean(y_train)
        y_pred = [avg_prediction] * len(y_test)
        mse, rmse, rsquared = evaluation_model(y_pred, y_test)
        mse_scores_avg.append(mse)
        rmse_list_avg.append(rmse)
        rsquared_list_avg.append(rsquared)
    print("------------------------------------------------------------")
    print("RMSE list: ", rmse_list) 
    print('\n')
    print("RSquared list: ", rsquared_list)
    print('\n')
    print("Mean RMSE in k-fold: ", np.mean(rmse_list))
    print("Mean RSquared in k fold: ", np.mean(rsquared_list))
    


In [ ]:
predictor_variable = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", 
          "ptb_eq5d_vas", "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", 
          "ptb_koospain", "fysb_9349", "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", 
          "ptb_3762", "ptb_13118", "fysb_medicin", "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b"]

##### Full model

In [ ]:
full_feature = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", 
          "ptb_eq5d_vas", "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", 
          "ptb_koospain", "fysb_9349", "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", 
          "ptb_3762", "ptb_13118", "fysb_medicin", "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b"]
kfold_training(data_vs, full_feature, target_column='vasChaSco', num_folds=10, random_state=42)
create_plot(data_vs, full_feature)

##### 11 variables

In [ ]:
feature = ["ptb_3764","ptb_eq5d_score_5l","testb_10391","fysb_symptomvarighed","fysb_BMI","ptb_eq5d_vas","age","ptb_koos_qol_score","testb_10281","pain_area","ptb_4145",]
kfold_training(data_vs, feature, target_column='vasChaSco', num_folds=10, random_state=42)
create_plot(data_vs, feature)

In [ ]:
len(feature)

#### For paper

##### Full model

In [ ]:
all = ["ptb_11836", "fysb_3639", "ptb_3764", "fysb_symptomvarighed", "ptb_eq5d_score_5l", "testb_10391", "fysb_BMI", "age", 
          "ptb_eq5d_vas", "pain_area", "testb_10281", "ptb_koos_qol_score", "ptb_4145", "ptb_3765", "ptb_10226_b", "ptb_16316", 
          "ptb_koospain", "fysb_9349", "fysb_3642", "ptb_low_back", "ptb_3792_b", "gender", "fysb_op", "ptb_10225", "ptb_3758", 
          "ptb_3762", "ptb_13118", "fysb_medicin", "ptb_3777_b", "ptb_3772", "ptb_10223", "testb_10392", "ptb_10224", "ptb_3754_b"]


kfold_training(data_vs, all, target_column='vasChaSco', num_folds=10, random_state=42)
create_plot(data_vs, all)

##### Top 11 variables

In [ ]:
feature = [
"ptb_3764","ptb_eq5d_score_5l","testb_10391","fysb_symptomvarighed","fysb_BMI", "ptb_eq5d_vas","age","ptb_koos_qol_score","testb_10281","pain_area","ptb_4145"]
kfold_training(data_vs, feature, target_column='vasChaSco', num_folds=10, random_state=42)
create_plot(data_vs, feature)

##### Selection model for web application / Concise model for clinical practice


In [ ]:
feature_with_speedWalk = ['age', 'fysb_BMI','ptb_3764','fysb_symptomvarighed', 'testb_10391', 'ptb_eq5d_score_5l']
kfold_training(data_vs, feature_with_speedWalk, target_column='vasChaSco', num_folds=10, random_state=42)
create_plot(data_vs, feature_with_speedWalk)

In [ ]:
feature_without_speedWalk = ['age', 'fysb_BMI','ptb_3764','fysb_symptomvarighed', 'ptb_eq5d_score_5l']
kfold_training(data_vs, feature_without_speedWalk, target_column='vasChaSco', num_folds=10, random_state=42)
create_plot(data_vs, feature_without_speedWalk)